In [186]:
import os
import folium
import geopandas as gpd
import pandas as pd
import json
import numpy as np
from shapely import wkt


In [187]:
# Read data from the xlsx file
grid_info = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "ficheiros_WDL", "DISPOSITIVOS MOVEIS_QUADRICULAS", "DISPOSITIVOS MOVEIS_QUADRICULAS.xlsx")
data = pd.read_excel(grid_info, sheet_name='em bruto', engine='openpyxl')

In [188]:
data.head()

,grelha_id,dicofre,entity_id,entity_type,freguesia,freguesias,grelha_x,grelha_y,latitude,longitude,nome,objectid,position,wkt
0,1,110658,coi.tematica.grelha.lx.atributos.3518,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,4,1,38.692094,-9.225027,Área ribeirinha Pedrouços,1,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.22412591616028 38.691391736...
1,2,110658,coi.tematica.grelha.lx.atributos.3519,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,5,1,38.692094,-9.223231,Área ribeirinha Pedrouços,2,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.22232928559204 38.691391736...
2,3,110658,coi.tematica.grelha.lx.atributos.3520,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,6,1,38.692094,-9.221434,Área ribeirinha Pedrouços,3,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.2205326550238 38.6913917367...
3,4,110658,coi.tematica.grelha.lx.atributos.3521,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,7,1,38.692094,-9.219637,Torre de Belém,4,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.21873602445556 38.691391736...
4,5,110658,coi.tematica.grelha.lx.atributos.3522,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,8,1,38.692094,-9.217841,Torre de Belém,5,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON {{{-9.21693939388732 38.691391736...


In [189]:
# Extract ids
ids = data["grelha_id"].tolist()

In [190]:
# Function to remove extra curly braces and return a valid WKT string
def fix_wkt_string(wkt_string):
    return wkt_string.replace("{{{", "(((").replace("}}}", ")))")

In [191]:
# Remove extra curly braces
data["wkt"] = data["wkt"].apply(fix_wkt_string)

In [192]:
# Apply wkt.loads
polygons = data["wkt"].apply(wkt.loads).tolist()

In [193]:
# Create a GeoDataFrame from the polygons
gdf = gpd.GeoDataFrame({"id": ids, "geometry": polygons})

In [194]:
# Calculate the center of the map (average latitude and longitude)
center_lat = gdf.geometry.centroid.y.mean()
center_lon = gdf.geometry.centroid.x.mean()

In [195]:
# Create a folium map
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

In [196]:
# Add grid squares to the map
for _, row in gdf.iterrows():
    folium.GeoJson(row.geometry, tooltip=f"ID: {row.id}").add_to(m)

In [197]:
# Save the map to an HTML file
m.save("grid_map.html")

# Add Estações_de_Comboio to data:

In [198]:
new_data = data.copy()
if 'nome_estacao' not in new_data:
    list(new_data.columns.values)
    new_data.insert(loc = len(new_data.columns),
            column = 'nome_estacao',
            value = " ")
    
if 'linha' not in new_data:
    new_data.insert(loc = len(new_data.columns),
            column = 'linha',
            value = 0)

In [199]:
# Target value
target = [-9.14609167401547, 38.7062703226346]

# Calculate squared Euclidean distances
aux = new_data.copy()
aux['distance'] = np.sqrt((new_data['longitude'] - target[0])**2 + (new_data['latitude'] - target[1])**2)

# Sort by distances
data_sorted = aux.sort_values('distance')

# Get row with smallest distance
closest_row = data_sorted.iloc[0, :]


In [200]:
# Specify the file path to GeoJSON data
file_path = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "redes", "Estações_de_Comboio.geojson")

# Read the file and load the data
with open(file_path, "r") as f:
    geojson_data = json.load(f)
    
line_color_dict = {}  # Dictionary to store line color mapping
features = geojson_data["features"]

for feature in features:
    coordinates = feature["geometry"]["coordinates"]
    nome = feature["properties"]["NOME"]
    objectid = feature["properties"]["OBJECTID"]
    linha = feature["properties"]["LINHA"]
    
    # Assign unique ID for different "LINHA" values
    if linha not in line_color_dict:
        line_color_dict[linha] = len(line_color_dict) + 1
    
    # Target value
    target = coordinates

    # Calculate squared Euclidean distances
    aux = new_data.copy()
    aux['distance'] = np.sqrt((new_data['longitude'] - target[0])**2 + (new_data['latitude'] - target[1])**2)

    # Sort by distances
    data_sorted = aux.sort_values('distance')

    # Get row with smallest distance
    closest_row = data_sorted.iloc[0, :]

    new_data.loc[new_data['grelha_id'] == closest_row['grelha_id'], 'linha'] = line_color_dict[linha]
    new_data.loc[new_data['grelha_id'] == closest_row['grelha_id'], 'nome_estacao'] = nome
    


In [201]:
new_data.head()


,grelha_id,dicofre,entity_id,entity_type,freguesia,freguesias,grelha_x,grelha_y,latitude,longitude,nome,objectid,position,wkt,nome_estacao,linha
0,1,110658,coi.tematica.grelha.lx.atributos.3518,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,4,1,38.692094,-9.225027,Área ribeirinha Pedrouços,1,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.22412591616028 38.691391736...,,0
1,2,110658,coi.tematica.grelha.lx.atributos.3519,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,5,1,38.692094,-9.223231,Área ribeirinha Pedrouços,2,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.22232928559204 38.691391736...,,0
2,3,110658,coi.tematica.grelha.lx.atributos.3520,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,6,1,38.692094,-9.221434,Área ribeirinha Pedrouços,3,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.2205326550238 38.6913917367...,,0
3,4,110658,coi.tematica.grelha.lx.atributos.3521,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,7,1,38.692094,-9.219637,Torre de Belém,4,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.21873602445556 38.691391736...,,0
4,5,110658,coi.tematica.grelha.lx.atributos.3522,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,8,1,38.692094,-9.217841,Torre de Belém,5,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.21693939388732 38.691391736...,,0


In [202]:

column_elements = new_data['linha'].unique().tolist()
print(column_elements)

linha_counts = new_data['linha'].value_counts()
print(linha_counts)

column_elements = new_data['nome_estacao'].unique().tolist()
print(column_elements)

linha_counts = new_data['nome_estacao'].value_counts()
print(linha_counts)

print("Dictionary Train Lines")
print(line_color_dict)


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
0     3694
2       15
1       10
4        9
9        9
3        1
5        1
6        1
7        1
8        1
10       1
Name: linha, dtype: int64
[' ', 'Cais do Sodré', 'Terreiro do Paço', 'Baixa Chiado', 'Rossio', 'Santa Apolónia', 'Restauradores', 'Martim Moniz', 'Rato', 'Avenida', 'Intendente', 'Marquês de Pombal', 'Anjos', 'Parque', 'Picoas', 'Arroios', 'São Sebastião', 'Saldanha', 'Alameda', 'Praça de Espanha', 'Olaias', 'Jardim Zoológico', 'Campo Pequeno', 'Areeiro', 'Entre Campos', 'Laranjeiras', 'Roma', 'Bela Vista', 'Alto dos Moinhos', 'Cidade Universitária', 'Reboleira', 'Colégio Militar', 'Alvalade', 'Chelas', 'Amadora Este', 'Alfornelos', 'Carnide', 'Telheiras', 'Campo Grande', 'Olivais', 'Pontinha', 'Cabo Ruivo', 'Quinta das Conchas', 'Aeroporto', 'Oriente', 'Lumiar', 'Encarnação', 'Moscavide', 'Ameixoeira', 'Odivelas']
                        3694
Telheiras                  1
Bela Vista                 1
Alto dos Moinhos           1
Cid

In [203]:
ids = new_data["grelha_id"].tolist()

def fix_wkt_string(wkt_string):
    return wkt_string.replace("{{{", "(((").replace("}}}", ")))")

new_data["wkt"] = new_data["wkt"].apply(fix_wkt_string)

polygons = new_data["wkt"].apply(wkt.loads).tolist()

# Convert linha column to integer to ensure correct color mapping
new_data['linha'] = new_data['linha'].astype(int)

gdf = gpd.GeoDataFrame({"id": ids, "geometry": polygons, "linha": new_data['linha'], "nome_estacao": new_data['nome_estacao']})

# Define color mapping based on linha column values
color_map_original = line_color_dict
print("color_map_original: " )
print(color_map_original)

# Swap the keys and values in the color mapping dictionary
color_map_swapped = {v: k for k, v in color_map_original.items()}
print("color_map_swapped: " )
print(color_map_swapped)

# Define a dictionary that maps linha values to colors
color_map = {1: 'lime', 2: 'blue', 3: 'cyan', 4: 'yellow', 5: 'green', 6: 'purple', 7: 'orange', 8: 'brown', 9: 'red', 10: 'olive'}
print("color_map: " )
print(color_map)

# Update the GeoDataFrame with the swapped color mapping
gdf['linha'] = gdf['linha'].map(color_map_swapped)


center_lat = gdf.geometry.centroid.y.mean()
center_lon = gdf.geometry.centroid.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# Set the CRS of gdf
gdf = gdf.set_crs("EPSG:4326")  # Replace "EPSG:4326" with the appropriate CRS for your data

# Function to style each feature
def style_function(feature):
    linha = feature['properties']['linha']
    if (linha==None):
        num_linha=0
    else:
        num_linha=color_map_original[linha]
    color = color_map.get(num_linha, 'gray')  # Use gray as default color for unknown values
    return {'fillColor': color, 'color': 'black', 'weight': 1, 'fillOpacity': 0.6}

# Add GeoJson layer to the map with style_function applied
folium.GeoJson(gdf, style_function=style_function, tooltip=folium.features.GeoJsonTooltip(fields=['id', 'nome_estacao'], aliases=['ID', 'Estação'])).add_to(m)

m.save("grid_map_station.html")


color_map_original: 
{'Verde': 1, 'Azul': 2, 'Azul/Verde': 3, 'Amarela': 4, 'Azul/Amarela': 5, 'Azul/Vermelha': 6, 'Amarela/Vermelha': 7, 'Verde/Vermelha': 8, 'Vermelha': 9, 'Amarela/Verde': 10}
color_map_swapped: 
{1: 'Verde', 2: 'Azul', 3: 'Azul/Verde', 4: 'Amarela', 5: 'Azul/Amarela', 6: 'Azul/Vermelha', 7: 'Amarela/Vermelha', 8: 'Verde/Vermelha', 9: 'Vermelha', 10: 'Amarela/Verde'}
color_map: 
{1: 'lime', 2: 'blue', 3: 'cyan', 4: 'yellow', 5: 'green', 6: 'purple', 7: 'orange', 8: 'brown', 9: 'red', 10: 'olive'}


# Add Rede_Ferroviária to data:

In [204]:
new_data_rede = data.copy()
if 'nome_linha' not in new_data_rede:
    list(new_data_rede.columns.values)
    new_data_rede.insert(loc = len(new_data_rede.columns),
            column = 'nome_linha',
            value = " ")
    
if 'segmentos_linha' not in new_data_rede:
    new_data_rede.insert(loc = len(new_data_rede.columns),
            column = 'segmentos_linha',
            value = " ")

In [205]:
# Specify the file path to GeoJSON data
file_path = os.path.join(os.path.dirname(os.getcwd()), "data", "raw", "redes", "Rede_Ferroviária.geojson")

# Read the file and load the data
with open(file_path, "r") as f:
    geojson_ferrovias = json.load(f)
    
linhas_dict = {}  # Dictionary to store line color mapping
features = geojson_ferrovias["features"]

for feature in features:
    coordinates = feature["geometry"]["coordinates"]
    nome = feature["properties"]["NOME"]
    objectid = feature["properties"]["OBJECTID"]
    linha = feature["properties"]["DESIG_SEGM"]
    
    # Assign unique ID for different "LINHA" values
    if linha not in linhas_dict:
        linhas_dict[linha] = len(linhas_dict) + 1
    
    # Target value
    #target = coordinates
    for target in coordinates:
        # Calculate squared Euclidean distances
        aux = new_data_rede.copy()
        aux['distance'] = np.sqrt((new_data_rede['longitude'] - target[0])**2 + (new_data_rede['latitude'] - target[1])**2)

        # Sort by distances
        data_sorted = aux.sort_values('distance')

        # Get row with smallest distance
        closest_row = data_sorted.iloc[0, :]

        new_data_rede.loc[new_data_rede['grelha_id'] == closest_row['grelha_id'], 'nome_linha'] = nome
        new_data_rede.loc[new_data_rede['grelha_id'] == closest_row['grelha_id'], 'segmentos_linha'] = linhas_dict[linha]
    

In [206]:
new_data_rede.head()

,grelha_id,dicofre,entity_id,entity_type,freguesia,freguesias,grelha_x,grelha_y,latitude,longitude,nome,objectid,position,wkt,nome_linha,segmentos_linha
0,1,110658,coi.tematica.grelha.lx.atributos.3518,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,4,1,38.692094,-9.225027,Área ribeirinha Pedrouços,1,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.22412591616028 38.691391736...,,
1,2,110658,coi.tematica.grelha.lx.atributos.3519,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,5,1,38.692094,-9.223231,Área ribeirinha Pedrouços,2,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.22232928559204 38.691391736...,,
2,3,110658,coi.tematica.grelha.lx.atributos.3520,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,6,1,38.692094,-9.221434,Área ribeirinha Pedrouços,3,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.2205326550238 38.6913917367...,,
3,4,110658,coi.tematica.grelha.lx.atributos.3521,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,7,1,38.692094,-9.219637,Torre de Belém,4,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.21873602445556 38.691391736...,,
4,5,110658,coi.tematica.grelha.lx.atributos.3522,coi.tematica.grelha.lx.atributos,Belém,Santa Maria de Belém + São Francisco Xavier,8,1,38.692094,-9.217841,Torre de Belém,5,"{'type': 'GeometryCollection', 'geometries': [...",MULTIPOLYGON (((-9.21693939388732 38.691391736...,,


In [207]:

column_elements = new_data_rede['nome_linha'].unique().tolist()
print(column_elements)

linha_counts = new_data_rede['nome_linha'].value_counts()
print(linha_counts)

column_elements = new_data_rede['segmentos_linha'].unique().tolist()
print(column_elements)

linha_counts = new_data_rede['segmentos_linha'].value_counts()
print(linha_counts)

print("Dictionary Train Lines")
print(linhas_dict)

[' ', 'Linha de Cascais', 'Linha do Sul', 'Linha de Cintura', 'Linha do Norte', 'Linha de Sintra', 'Linha da Matinha', 'Concordância de Sete Rios']
                             3434
Linha de Cintura               78
Linha do Norte                 75
Linha de Sintra                61
Linha de Cascais               58
Linha do Sul                   19
Linha da Matinha               17
Concordância de Sete Rios       1
Name: nome_linha, dtype: int64
[' ', 6, 1, 2, 4, 9, 7, 5, 3, 8]
     3434
4      74
7      61
9      60
6      58
1      19
5      17
8      15
2       4
3       1
Name: segmentos_linha, dtype: int64
Dictionary Train Lines
{'Agulha junto à Ponte Santana / Pinhal Novo': 1, 'Alcântara Mar / Alcântara Terra': 2, 'Sete Rios / Benfica': 3, 'Alcântara Terra / Braço de Prata': 4, 'Lisboa Santa Apolónia / Lisboa-Matinha': 5, 'Cais do Sodré / Cascais': 6, 'Lisboa Rossio / Sintra': 7, 'Lisboa Santa Apolónia / Porto Campanhã/ Parque Nações/ Loures': 8, 'Lisboa Santa Apolónia / Porto C